# mesures_cat_seuil
***

Essayons de mettre en pratique la proposition de Thomas dans la PR 89<br>
https://github.com/dataforgoodfr/13_pollution_eau/pull/89


# Proposition
***
Je pense qu'il serait intéressant d'avoir le stables intémédiares suivantes<br>
1. une table pour les données géo : comme udi_cog<br>
2. une table un par test/resultat avec les regexp et les catégories<br>

/!\ Pour le traitement des données par région/commune/departement, il ne faut pas compter les resultats ou faire des pourcentages car un même résultat peut être utilisés plusieurs fois.
Il faudra juste avoir des données plus globale sur la qualité de l'eau : conforme, avec/sans pestidices, etc...

3. une autre table avec les données par communes<br>
4. une autre table avec les données par depatement <br>
5. une autre table avec les données par région<br>
6. une autre table avec les données par UDI<br>


# Questions ouvertes
***

**Dans edc_communes, un cdreseau peut être lié à plusieurs communes et inversement.<br>
Est-ce attendu ?**<br>

Si je relis la partie "DESCRIPTIF DES DONNEES MISES A DISPOSITION" du pdf :<br>
" Les UDI peuvent englober les réseaux de distribution d’eau dans tout ou partie d’une ou plusieurs communes, suivant la situation locale.<br>
[...] Le fichier UDI_COM détaille le lien entre UDI et communes et permet de pouvoir reconstituer un accès aux résultats par commune ; il est relié au fichier PLV par l’intermédiaire du champ ‘cdreseau’, correspondant au code SISE-Eaux de distribution."
<br> ==> J'ai l'impression que oui


**Si on fait les jointures comme recommandé dans la documentation pour avoir les informations de communes/géographie,
on duplique les nombres de lignes de résultats. 
Un même résultats/prélèvement peut être utilisé pour plusieurs communes.**<br>
--> Est-ce qu'on est ok avec ça ?

_Dans la table edc_resultats on a 64 164 498 lignes_

Voici différents tests
- sub_query = Check avant jointure par cdreseau en utilisant prelevement_unique : même nombre de lignes que edc_resultats
- sub_query2 = Check avant jointure par cdreseau en utilisant edc_prelevement : 33 315 337 lignes en plus
- query = sub_query + Jointe par cdreseau sur communes : 581 458 710 lignes en plus
- query2 = sub_query + Jointe par cdreseau sur communes : 1 784 283 587 lignes en plus


Pour l'analyse des données de prélèvement est-ce qu'on veut garder tous les cdreseaux ou le plus en amont/ le "first" suffit ?<br>
--> Je pense qu'on veut garder toutes les lignes cdreseau

**Je pense que la table intermédiare que l'on souhaite avec les info par communes est la query2 mais il faut peut-être traiter les informations avec prudence car on a dupliquer des résultats (du à la situation locale)**


In [1]:
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE

import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [2]:
# Tables dispo dans la bdd
con.sql("SHOW TABLES").show()

┌──────────────────────────────┐
│             name             │
│           varchar            │
├──────────────────────────────┤
│ ana__resultats_communes      │
│ cog_communes                 │
│ edc_communes                 │
│ edc_prelevements             │
│ edc_resultats                │
│ int__mapping_category_simple │
│ int__prelevements_uniques    │
│ laposte_communes             │
│ mapping_categories           │
│ stg_communes__cog            │
│ stg_communes__laposte        │
│ stg_edc__communes            │
│ stg_edc__prevelevements      │
│ stg_edc__resultats           │
├──────────────────────────────┤
│           14 rows            │
└──────────────────────────────┘



# Proposition de nouvelle table intermédiaire
***

In [ ]:
query_mesures_cat = """
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    )
    
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
"""

# df_mesures_cat = con.sql(query_mesures_cat).df() -- Pas assez de mémoire sur mon ordi

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_12596\2272530417.py:1: SyntaxWarning: invalid escape sequence '\d'
  query_mesures_cat = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query_udi_cog = """
WITH
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
)
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
"""

df_udi_cog = con.sql(query_udi_cog).df()

# Exploration
***

In [ ]:
nb_result_ref = con.sql(" SELECT COUNT(*) FROM edc_resultats").df()
nb_result_ref

## 1. Check avant jointure par cdreseau en utilisant prelevement_unique

In [ ]:
sub_query = """
WITH
prelevement AS (
	SELECT
	  referenceprel,
	  cdfirstreseauamont AS cdreseau,
      dateprel,
	FROM   
      int__prelevements_uniques    
),
    
mesures_cat AS (
    SELECT
        df_mesures_cat.*,
        prelevement.dateprel,
    FROM 
        df_mesures_cat
    LEFT JOIN
    	prelevement
    ON
       df_mesures_cat.referenceprel = prelevement.referenceprel
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_sub_query = con.sql(sub_query).df()

if nb_result_sub_query.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_sub_query.iloc[0].values - nb_result_ref.iloc[0].values)

## 2. Check avant jointure par cdreseau en utilisant edc_prelevement

In [ ]:
sub_query2 = """
WITH
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

mesures_cat AS (
    SELECT
        df_mesures_cat.*,
        prelevement.dateprel,
        int__mapping_category_simple.categorie
    FROM 
        df_mesures_cat
    LEFT JOIN
    	prelevement
    ON
       df_mesures_cat.referenceprel = prelevement.referenceprel
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_sub_query2 = con.sql(sub_query2).df()

if nb_result_sub_query2.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_sub_query2.iloc[0].values - nb_result_ref.iloc[0].values)

## 3. Avec Join sur edc_communes et cog_communes à partir du resultat 1

In [ ]:
query = """
WITH
prelevement AS (
	SELECT
	  referenceprel,
	  cdfirstreseauamont AS cdreseau,
      dateprel,
	FROM   
      int__prelevements_uniques    
),

mesures_cat AS (
    SELECT
        df_mesures_cat.*,
        prelevement.dateprel,
        int__mapping_category_simple.categorie
    FROM 
        df_mesures_cat
    LEFT JOIN
    	prelevement
    ON
       df_mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        df_udi_cog
    ON
    	df_udi_cog.cdreseau = prelevement.cdreseau
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_1 = con.sql(query).df()

if nb_result_1.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_1.iloc[0].values - nb_result_ref.iloc[0].values)

## 4. Avec Join sur edc_communes et cog_communes à partir du resultat 2

In [ ]:
query2 = """
WITH
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

mesures_cat AS (
    SELECT
        df_mesures_cat.*,
        prelevement.dateprel,
        int__mapping_category_simple.categorie
    FROM 
        df_mesures_cat
    LEFT JOIN
    	prelevement
    ON
       df_mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        df_udi_cog
    ON
    	df_udi_cog.cdreseau = prelevement.cdreseau
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_2 = con.sql(query2).df()

if nb_result_2.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_2.iloc[0].values - nb_result_ref.iloc[0].values)

# ZOOM uid/cog

In [ ]:
query_udi_cog = """  
SELECT 
 cdreseau,
 COUNT(*)
FROM
 df_udi_cog
GROUP BY 1
ORDER BY 2 DESC
"""

udi_cog_df = con.sql(query_udi_cog).df()
udi_cog_df

In [ ]:
query_udi_cog = """    
SELECT 
 cdreseau,
 COUNT(*)
FROM
 df_udi_cog
GROUP BY 1
ORDER BY 2 DESC
"""

udi_cog_df = con.sql(query_udi_cog).df()
udi_cog_df

In [ ]:
query_test = """
SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
   WHERE 
      cdreseau = '048000295'
"""
con.sql(query_test).df()

In [ ]:
query_udi_cog = """   
SELECT 
 commune_code_insee,
 COUNT(cdreseau)
FROM
 df_udi_cog
GROUP BY 1
ORDER BY 2 DESC
"""

udi_cog_df = con.sql(query_udi_cog).df()
udi_cog_df

In [ ]:
query_test = """
SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
   WHERE 
      commune_code_insee = '48009'
"""
con.sql(query_test).df()